In [14]:
# Setup stuff

import google.generativeai as genai

# api key stuff

import os
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

c:\Users\Sajiv\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from autokattis import Kattis
import pandas as pd
import os

user = os.getenv('KATTIS_USER')
pwd = os.getenv('KATTIS_PASS')
kt = Kattis(user, pwd)

Candidate username(s): ['sajiv']
Successfully logged in to Kattis!


In [3]:
# model test
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')
prompt = ""
response = model.generate_content("What is Google?")

In [4]:
response.text

'**Google LLC**\n\n**Overview:**\n\n* A multinational technology company specializing in Internet-related products and services.\n* Founded in 1998 by Larry Page and Sergey Brin.\n* Headquartered in Mountain View, California, United States.\n\n**Core Businesses:**\n\n**Search Engine:**\n\n* Google Search: Provides results for text, image, video, and other content on the web.\n* Google Maps: Provides detailed maps and navigation tools.\n* Google Earth: Delivers 3D representations of the Earth\'s surface.\n\n**Advertising:**\n\n* Google Ads: An advertising platform for businesses to reach targeted audiences.\n* Google AdSense: A program that allows website owners to earn revenue from displaying ads.\n\n**Cloud Computing:**\n\n* Google Cloud: Offers a wide range of cloud computing services, including storage, databases, and machine learning.\n\n**Software and Hardware:**\n\n* Android Operating System: Used in over 80% of smartphones worldwide.\n* Chrome Browser: A fast and secure web brow

In [23]:
# rerun to update subfolders variables
import os

# Specify the path of the base directory
base_dir = 'kattis2'

# Get all entries in the directory
entries = os.listdir(base_dir)

# Filter out the entries to keep only directories
subfolders = [entry for entry in entries if os.path.isdir(os.path.join(base_dir, entry))]

# Count the subfolders
subfolder_count = len(subfolders)
print(f"Total subfolders: {subfolder_count}")

Total subfolders: 3705


In [25]:
def extract_code_to_file(code_block, filename='gemini.py'):
    # Split the block by lines
    lines = code_block.split('\n')
    # Remove the first and last lines which are the markdown code block markers
    stripped_lines = lines[1:-1]
    # Join the lines back into a single string
    code = '\n'.join(stripped_lines)
    
    # Write the extracted code to a file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(code)
    
    print(f"Code has been written to {filename}")

In [5]:
# trying out on the first 3 files

# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')

responses = []

for subfolder in subfolders[:3]:

    print(subfolder)

    file_path = 'kattis2/' + subfolder
    # Open the file and read its contents into a string
    with open(file_path + '/prompt', 'r', encoding='utf-8') as file:
        prompt = file.read()

    response = model.generate_content("Provide the python code for the following prompt. Do not give sample data. Assume that data will be given as user input " + prompt)
    responses.append(response.text)
    extract_code_to_file(response.text, file_path + '/gemini.py')


10kindsofpeople
Code has been written to kattis2/10kindsofpeople/gemini.py
1dfroggereasy
Code has been written to kattis2/1dfroggereasy/gemini.py
1dfroggerhard
Code has been written to kattis2/1dfroggerhard/gemini.py


In [22]:
# don't run again, first three are done
# grade first three problems and save string with line needed
import os
import subprocess

for subfolder in subfolders[:3]:

    print(subfolder)

    # get difficulty for problem
    difficulty = kt.problem(subfolder)[0]['difficulty']
    appString = str(difficulty)

    # get grade, parse output
    grade = grade_report(subfolder)
    appString = appString + " " + parse_grade(grade)
    print(appString)
    # append string to top of file
    add_comment_to_top(subfolder, appString)



    

10kindsofpeople
4.6 Error 1/25
1dfroggereasy
3.5 Error 1/38
1dfroggerhard
8.6 Error 1/12


In [19]:
def add_comment_to_top(problem, comment: str) -> None:
    """
    Add a comment at the top of a Python file.

    Args:
        filepath (str): The path to the Python file to be modified.
        comment (str): The comment string to add, without the "#" character.
    """

    filepath = os.path.join('kattis2', problem, 'gemini.py')
    
    # Prepare the comment string
    comment_line = f"# {comment}\n"

    # Read the original file contents
    with open(filepath, 'r') as file:
        original_contents = file.read()

    # Write the new contents to the file (prepend the comment)
    with open(filepath, 'w') as file:
        file.write(comment_line + original_contents)

# Example usage:
# add_comment_to_top('example.py', 'This is a new comment added at the top of the file')


In [11]:
import os
import re
import time
import subprocess

def grade_report(problem):


    def run_command():
        """Run the Kattis command and return the output."""
        file_path = os.path.join('kattis2', problem, 'gemini.py')
        command = ['kattis', file_path, '-p', problem]

        # Run the command and capture the output
        process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True)
        process.communicate(input='y\n')

        # Capture the output and error streams
        output, error = process.communicate()
        return output

    while True:
        # Run the command and get the output
        output = run_command()

        # Check if output indicates "out of submission tokens"
        match = re.search(r'Your next token will regenerate in (\d+) seconds', output)
        if match:
            # Extract the wait time in seconds
            wait_time = int(match.group(1))
            # wait a little longer
            wait_time += 10
            print(f"Out of submission tokens. Waiting {wait_time} seconds before retrying...")
            time.sleep(wait_time)
        else:
            # If no waiting message found, return the output
            return output


In [12]:
# parse grade
import re

def parse_grade(output: str) -> str:
    # Extract the status and number of test cases passed
    test_case_pattern = re.search(r'Test cases:.*\[(.*?)\].* (\d+) / (\d+)', output)
    if test_case_pattern:
        passed_tests = int(test_case_pattern.group(2))
        total_tests = int(test_case_pattern.group(3))
        test_status = output.splitlines()[-1].split(' ')[0]
        
        # Determine the result message
        if test_status == "Accepted":
            result = "right answer"
        elif test_status == "Wrong":
            result = "wrong answer"
        else:
            result = "Error"
        
        # Return formatted result string
        return f"{result} {passed_tests}/{total_tests}"
    else:
        return "Error 0/0"

In [35]:
# get gemini code for prompts 4-99 ended on amsterdamdistance
# re running 5/6/23 @3pm to update gemini responses up to 1-100 in subfolders[]

# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')


for subfolder in subfolders[19:100]:

    print(subfolder)

    file_path = 'kattis2/' + subfolder
    # Open the file and read its contents into a string
    with open(file_path + '/prompt', 'r', encoding='utf-8') as file:
        prompt = file.read()

    response = model.generate_content("Provide the python code for the following prompt. Do not give sample data. Assume that data will be given as user input " + prompt)
    extract_code_to_file(response.text, file_path + '/gemini.py')


abstractart
Code has been written to kattis2/abstractart/gemini.py
abstractpainting
Code has been written to kattis2/abstractpainting/gemini.py
absurdistan2
Code has been written to kattis2/absurdistan2/gemini.py
absurdistan3
Code has been written to kattis2/absurdistan3/gemini.py
acappellarecording
Code has been written to kattis2/acappellarecording/gemini.py
acceptableseatingarrangements
Code has been written to kattis2/acceptableseatingarrangements/gemini.py
accessdenied
Code has been written to kattis2/accessdenied/gemini.py
accesspoints
Code has been written to kattis2/accesspoints/gemini.py
acearbiter
Code has been written to kattis2/acearbiter/gemini.py
acm
Code has been written to kattis2/acm/gemini.py
acm2
Code has been written to kattis2/acm2/gemini.py
acomplexproblem
Code has been written to kattis2/acomplexproblem/gemini.py
ada
Code has been written to kattis2/ada/gemini.py
adaloveslaces
Code has been written to kattis2/adaloveslaces/gemini.py
addemup
Code has been written 

In [34]:
subfolders[18]

'absolutelysymmetric'

In [36]:
# run above cell, update first 100 gemini python responses
# then run this cell to grade those responses
import os
import subprocess


for subfolder in subfolders[19:100]:

    print(subfolder)

    # get difficulty for problem
    difficulty = kt.problem(subfolder)[0]['difficulty']
    appString = str(difficulty)

    # get grade, parse output
    grade = grade_report(subfolder)
    appString = appString + " " + parse_grade(grade)
    print(appString)
    # append string to top of file
    add_comment_to_top(subfolder, appString)

abstractart
6.9 Error 1/22
abstractpainting
4.2 Error 1/58
absurdistan2
6.8 Error 1/18
absurdistan3
5.8 Error 1/22
acappellarecording
3.6 Error 2/17
acceptableseatingarrangements
6.9 Error 1/9
accessdenied
4.1 Error 0/92
accesspoints
5.8 Error 1/50
acearbiter
5.0 Error 1/40
acm
2.1 Error 1/7
acm2
2.7 Error 1/11
acomplexproblem
5.8 Error 1/24
ada
2.4 Error 1/17
adaloveslaces
4.7 Error 0/0
addemup
5.1 Error 0/0
addingtrouble
1.4 Error 0/0
addingwords
4.1 Error 0/0
additiongame
8.7 Error 0/0
addormultiply
7.9 Error 0/0
addtwonumbers
1.3 Error 0/0
adjoin
5.8 Error 0/0
administrativeproblems
5.8 Error 0/0
adolescentarchitecture
4.4 Error 0/0
adventurebegins
6.0 Error 0/0
adventuremoving4
5.4 Error 0/0
advertisingicpc
8.0 Error 0/0
agamemnonsodyssey
4.8 Error 0/0
agario
8.5 Error 0/0
agglomerator
4.4 Error 0/0
ahorseshoein
9.0 Error 0/0
aijeopardy


KeyboardInterrupt: 